<font color="yellow"><h1><big><big><big><big><big><big>NOTE TO JARVIS MEMBERS:</big></big></big></big></big></big></h1>
<small><small>and maybe sir Henry <sub>(For help)</sub><sup>Hi sir!</sup></small></small><br><br>
<big><big><big><big>
TODO:
</big></big></big></big><big><big><big><big>Implement the model scoring and evaluation scores</big></big></big></big></font>

<h1>Fine-tuning BERT (and friends) for multi-label text classification</h1>

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.


<h2> Set-up environment</h2>

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

# Installation

In [ ]:
"""
JARVIS NOTE:  Models are saved in Razi's Google Drive
              Feel free to save it in your own folder if you'd like
              If you don't have access... you're forced to pretrain :)

LINKS      :  https://drive.google.com/drive/folders/1vgTzNAQWJv47eW0FWK8uHbhk9wPdTQhZ?usp=share_link [DATASETS]
              https://drive.google.com/drive/folders/1q9jPZz253xMQ96gs2DICccRY44yXEe1k?usp=sharing [SAVED PRETRAINED MODEL]

MODEL NAMES TO BE PASSED INTO THE ARGUMENTS
BERT Base             : bert-base-uncased
IndoBERT              : indolem/indobert-base-uncased
Indonesian DistilBERT : cahya/distilbert-base-indonesian
Multilingual BERT     : bert-base-multilingual-cased
"""
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.8 MB/s eta 0:00:00


In [ ]:
""" JARVIS' MODIFICATION ⬇️ """
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 12.4 MB/s eta 0:00:00


In [ ]:
""" JARVIS' MODIFICATION ⬇️ """
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Preparing The Data

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [ ]:
from datasets import load_dataset

""" Default Dataset: """
# dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")

""" Twitter Dataset: [SAVED IN RAZI'S GOOGLE DRIVE] """
# dataset = load_dataset("csv", data_files={'train': '/content/drive/MyDrive/Sharing/Datasets/data.csv', 'test': '/content/drive/MyDrive/Sharing/Datasets/data.csv', 'validation': '/content/drive/MyDrive/Sharing/Datasets/data.csv'}, encoding='latin-1')

""" SYNTAX FOR LOADING THE DATASET """
# dataset = load_dataset("csv", data_files={'train': 'YourDirectory', 'test': 'YourDirectory', 'validation': 'YourDirectory'}, encoding='latin-1')
dataset = load_dataset("csv", data_files={'train': '/content/drive/MyDrive/Dataset/data.csv', 'test': '/content/drive/MyDrive/Dataset/data.csv', 'validation': '/content/drive/MyDrive/Dataset/data.csv'}, encoding='latin-1')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9f32f0f09237dfa6/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Tweet', 'HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak', 'HS_Moderate', 'HS_Strong'],
        num_rows: 13169
    })
    test: Dataset({
        features: ['Tweet', 'HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak', 'HS_Moderate', 'HS_Strong'],
        num_rows: 13169
    })
    validation: Dataset({
        features: ['Tweet', 'HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak', 'HS_Moderate', 'HS_Strong'],
        num_rows: 13169
    })
})

Let's check the first example of the training split:

In [ ]:
example = dataset['train'][0]
example

{'Tweet': "- disaat semua cowok berusaha melacak perhatian gue. loe lantas remehkan perhatian yg gue kasih khusus ke elo. basic elo cowok bego ! ! !'",
 'HS': 1,
 'Abusive': 1,
 'HS_Individual': 1,
 'HS_Group': 0,
 'HS_Religion': 0,
 'HS_Race': 0,
 'HS_Physical': 0,
 'HS_Gender': 0,
 'HS_Other': 1,
 'HS_Weak': 1,
 'HS_Moderate': 0,
 'HS_Strong': 0}

The dataset consists of tweets, labeled with one or more emotions.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['HS',
 'Abusive',
 'HS_Individual',
 'HS_Group',
 'HS_Religion',
 'HS_Race',
 'HS_Physical',
 'HS_Gender',
 'HS_Other',
 'HS_Weak',
 'HS_Moderate',
 'HS_Strong']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [ ]:
from transformers import AutoTokenizer
import numpy as np

"""
JARVIS NOTE: Change this to the appropriate token

MODEL NAMES TO BE PASSED INTO THE ARGUMENT
BERT Base             : bert-base-uncased
IndoBERT              : indolem/indobert-base-uncased
Indonesian DistilBERT : cahya/distilbert-base-indonesian
Multilingual BERT     : bert-base-multilingual-cased
"""
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["Tweet"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/13169 [00:00<?, ? examples/s]

Map:   0%|          | 0/13169 [00:00<?, ? examples/s]

Map:   0%|          | 0/13169 [00:00<?, ? examples/s]

In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

"[CLS] - disaat semua cowok berusaha melacak perhatian gue. loe lantas remehkan perhatian yg gue kasih khusus ke elo. basic elo cowok bego!!!'[SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [ ]:
example['labels']

[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['HS', 'Abusive', 'HS_Individual', 'HS_Other', 'HS_Weak']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [ ]:
encoded_dataset.set_format("torch")

# Model Setup

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [ ]:
from transformers import AutoModelForSequenceClassification

"""
JARVIS NOTE: Change the argument that stores the model type to the appropriate model

MODEL NAMES TO BE PASSED INTO THE ARGUMENT
BERT Base             : bert-base-uncased
IndoBERT              : indolem/indobert-base-uncased
Indonesian DistilBERT : cahya/distilbert-base-indonesian
Multilingual BERT     : bert-base-multilingual-cased
"""
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer
""" JARVIS' MODIFICATION ⬇️ """
from accelerate.state import PartialState
"""
PROBLEM       : TrainingArguments class has an undefined class 'PartialState'
                from the 'accelerate' library
                Check 'training_args.py' for more details:
                /usr/local/lib/python3.10/dist-packages/transformers/training_args.py in _setup_devices(self)

FIX ATTEMPT #1: Installing and upgrading the accelerate library using
                pip install --upgrade accelerate
                [FAILED]
FIX ATTEMPT #2: Removing if condition for the 'is_accelerate_available()'
                function to force an import
                [FAILED]
FIX ATTEMPT #3: Importing 'PartialState' directly into the program
                [SOLVED]
"""
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-7,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

# Define The Model <font color="yellow">PICK ONE: {PRELOADED / NEW MODEL}</font>

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

## Train New Model <font color="red">[30 MINUTES~]</font>

Let's verify a batch as well as a forward pass:

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([   101,    118,  36474,  10526,  23367,  11170,  16828,  10174,  81825,
         10911,  63279,  10710, 103601,  75980,  10112,    119,  10406,  10112,
         12228,  11390,  11639,  10627,  25015, 103601,    193,  10240,  75980,
         10112,  14399,  13187,  48147,  11163,  10125,  10133,    119,  25090,
         10125,  10133,  11170,  16828,  10174,  10347,  10797,    106,    106,
           106,    112,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7031, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0132, -0.1760, -0.2057, -0.0218,  0.0870,  0.0530, -0.0039, -0.1907,
          0.2216,  0.0563,  0.0896,  0.0630]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [ ]:
trainer = Trainer(
    model,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.364900
1000,0.308900
1500,0.281000
2000,0.257300
2500,0.238400
3000,0.230100
3500,0.208400
4000,0.177400
4500,0.170500


TrainOutput(global_step=4941, training_loss=0.24195875095660038, metrics={'train_runtime': 1165.6823, 'train_samples_per_second': 33.892, 'train_steps_per_second': 4.239, 'total_flos': 2598915441208320.0, 'train_loss': 0.24195875095660038, 'epoch': 3.0})

# Evaluate

After training, we evaluate our model on the validation set.

In [ ]:
trainer.evaluate()

{'eval_loss': 0.13833826780319214,
 'eval_f1': 0.8553932584269662,
 'eval_roc_auc': 0.9040528487913343,
 'eval_accuracy': 0.7701420001518718,
 'eval_runtime': 95.4767,
 'eval_samples_per_second': 137.929,
 'eval_steps_per_second': 17.25,
 'epoch': 3.0}

# Inference

Let's test the model on a new sentence:

In [ ]:
text = "Bacot kau"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 12])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['HS', 'Abusive', 'HS_Individual', 'HS_Other', 'HS_Weak']


# Saving Pretrained Model <font color="red">[RUN ONLY ONCE]</font>

In [ ]:
"""
Saving pretrained model
IMPORTANT!: Save in proper directory (Change if needed)
"""

""" BERT Base storage """
# model.save_pretrained('/content/drive/MyDrive/BERT Models/bert-base-uncased')

""" IndoBERT storage """
# model.save_pretrained('/content/drive/MyDrive/BERT Models/indobert-base-p1')

""" Indo DistilBERT storage """
# model.save_pretrained('/content/drive/MyDrive/BERT Models/cahya distilbert-base-indonesian')

""" Multilingual-BERT storage """
# model.save_pretrained('/content/drive/MyDrive/BERT Models/cahya distilbert-base-indonesian')

' Multilingual-BERT storage '